# Jan 2019 New functionalities examples

Example demonstrating how to use the torque_tube functionality.

#### Prerequisites (Step 0):
We assume all instalation is working at this point, reffer to the other journals for this! 

# Torque Tube Example (Hex!)

In [3]:
import sys

sys.path.insert(0, '../bifacial_radiance')

try:
    from bifacial_radiance import *
    print " Successful import of bifacial_radiance version . "
except ImportError:
    raise RuntimeError('bifacial_radiance is required. download distribution')
    # Simple example system using Radiance.

 Successful import of bifacial_radiance version . 


In [ ]:
# The below routine creates a HEXAGONAL torque tube, for a 2-UP configuration of a specific module size. Parameters for the module, the torque tube, and the scene are below.
# This is being run with gendaylit, for one specific timestamp

timestamp = 4020 # Noon, June 17th. 
simulationname = 'Torque_tube_hex_test'

# MakeModule Parameters
module_type='my_custom_panel'
module_length = 1.996  # 2-up portrait Longi with 15cm additional gap
module_width = 0.991
orientation='portrait' 
tilt = 10

# TorqueTube Parameters
torqueTube = True
numpanels = 2
gap = 0.15
tubetype = 'Hex'
disttopanel = 0.175
diameter = 0.15
torqueTubeMaterial = 'Metal_Grey'       # IT's NOT GRAY, IT's GREY

# SceneDict Parameters
gcr = 0.33   # ground cover ratio,  = module_height / pitch
albedo = 0.28  #'concrete'     # ground albedo
clearance_height = 2.35   
azimuth_ang=90
sensorsy = 200 # Notice we will be doing 200 sensors! It takes more time but gives better resolution.
psx = 0.05

In [ ]:
# MODIFY THE TEST FOLDER TO YOUR LIKING. I utmostly dislike the interactive directory.
testfolder = r'C:\Users\sayala\Documents\RadianceScenes\Octtest'

In [ ]:
# Now let's run the example

demo = RadianceObj('Torque_tube_hex_test',path = testfolder)  # Create a RadianceObj 'object'
demo.setGround(0.62) # input albedo number or material name like 'concrete'.  To see options, run this without any input.
epwfile = demo.getEPW(37.5,-77.6) # pull TMY data for any global lat/lon
metdata = demo.readEPW(epwfile) # read in the EPW weather data from above
demo.gendaylit(metdata,4020)  # Noon, June 17th

#We have a 2-up configuration in portrait. This section helps sort what is the COLLECTOR WIDTH to use.
if orientation == 'portrait':
                slope=module_height
elif orientation == 'landscape':  
                slope=module_width
collectorwidth = slope*numpanels+gap*(numpanels-1) # If it's 1UP, the gap value will not matter. This is a safety.

# Making module with all the variables
demo.makeModule(name=module_type,x=module_width,y=module_length,bifi=1,orientation='portrait', 
           torquetube=torqueTube, diameter = diameter, tubetype = tubetype, material = torqueTubeMaterial, tubeZgap = disttopanel, numpanels = numpanels, panelgap = gap, rewriteModulefile = True, psx=psx)
# create a scene with all the variables
sceneDict = {'tilt':tilt,'pitch': np.round(collectorwidth / gcr,3),'height':clearance_height,'orientation':orientation,'azimuth':azimuth_ang, 'module_type':module_type}  

scene = demo.makeScene(moduletype=module_type, sceneDict=sceneDict, nMods = 20, nRows = 7, sensorsy = sensorsy, psx = psx) #makeScene creates a .rad file with 20 modules per row, 7 rows.
octfile = demo.makeOct(demo.getfilelist())  # makeOct combines all of the ground, sky and object files into a .oct file.


At this point you should be able to go into a command window (cmd.exe) and check the geometry. Example:
    
   ## rvu -vf views\front.vp -e .01 Torque_tube_hex_test.oct
   
   And then proceed happily with your analysis:

In [ ]:
analysis = AnalysisObj(octfile, demo.name)  # return an analysis object including the scan dimensions for back irradiance
analysis.analysis(octfile, demo.name, scene.frontscan, scene.backscan)  # compare the back vs front irradiance  
print('Annual bifacial ratio average:  %0.3f' %( sum(analysis.Wm2Back) / sum(analysis.Wm2Front) ) )


And then you dance-dance-dance. Except that, at the current stage of the software, if you open the results you will notice that now some of the sensors are measuring the irradinace at the torque tube and must die... in a future version.

(or increase the number of sensors by a lot and take them down in a separate routine in matlab all pretily and interpolate to have a consistent number of sensors with data while single axis tracking).

# Single-axis-tracking by HOUR example

Think of this as a fixed-tilt where the clearance height and the tracker tilt is changing. So the angles have to be re-calculated for every timestamp.

In [ ]:
#Some variables again. Assuming bifacial_radiance was imported.

timeindex = 4020 # Noon, June 17th. 
simulationname = '1axisTrack'+str(timeindex)

# MakeModule Parameters
module_type='my_little_pony_panel'
module_length = 1.996  # 2-up portrait Longi with 15cm additional gap
module_width = 0.991
orientation='portrait' 
tilt = 10

# Tracking Angle Calculation Parameters
hub_height = 2.35   
axis_azimuth = 180.0
roundTrackerAngleBool = True 
angledelta = 5
limit_angle = 45
backtrack = True
    
# SceneDict Parameters
gcr = 0.33   # ground cover ratio,  = module_height / pitch
albedo = 0.28  #'concrete'     # ground albedo

In [ ]:
#separate folder for this example
testfolder = r'C:\Users\sayala\Documents\RadianceScenes\asdf2'

In [ ]:
# Running the simulation
demo2 = RadianceObj(simulationname, path = testfolder)  # Create a RadianceObj 'object' named 'demo'
demo2.setGround(albedo) # input albedo number or material name like 'concrete'.  To see options, run this without any input.
epwfile = demo2.getEPW(37.5,-77.6) # pull TMY data for any global lat/lon
metdata = demo2.readEPW(epwfile) # read in the weather data
demo2.gendaylit(metdata, 4020)  # Noon, June 17th

demo2.makeModule(name=module_type,x=module_width,y=module_length)    
tracker_theta, tracker_height, azimuth_ang = demo2._getTrackingGeometryTimeIndex(metdata, timeindex=timeindex, angledelta = angledelta, roundTrackerAngleBool = roundTrackerAngleBool, axis_azimuth = axis_azimuth, limit_angle = limit_angle, backtrack = backtrack, gcr = gcr, hubheight = hub_height, module_height = module_height )
sceneDict = {'tilt':tracker_theta,'pitch': module_height / gcr,'height':tracker_height,'orientation':orientation,'azimuth':azimuth_ang, 'module_type':module_type}  
scene = demo2.makeScene(module_type, sceneDict) #makeScene creates a .rad file with 20 modules per row, 7 rows.
octfile = demo2.makeOct(demo2.getfilelist())  # makeOct combines all of the ground, sky and object files into a .oct file.

At this point you should be able to go into a command window (cmd.exe) and check the geometry. Example:
    
   ## rvu -vf views\front.vp -e .01 Torque_tube_hex_test.oct
   
   And then proceed happily with your analysis:

In [ ]:
analysis = AnalysisObj(octfile, demo2.basename)  # return an analysis object including the scan dimensions for back irradiance
frontDict, backDict = analysis.analysis(octfile, demo2.basename, scene.frontscan, scene.backscan)  # compare the back vs front irradiance  
print('Annual bifacial ratio: %0.3f - %0.3f' %(min(analysis.backRatio), np.mean(analysis.backRatio)) )

After you set up your system parameters, you can just change the timestamp and evaluate any day. You could also use gendaylit2manual to pass specific DNI/DHI if you so wanted.

Enjoy!

# Add Custom Elements to your Scene
This shows how to add a custom element, in this case a Cube, that will be placed in the center of your scene to mark the 0,0 location. A tracking-example with torque tube is used.

In [18]:
#First all the Variables:
testfolder = r'C:\Users\sayala\Documents\RadianceScenes\demo'

#Some variables again. Assuming bifacial_radiance was imported.

timeindex = 4020 # Noon, June 17th. 
simulationname = '1axisTrack'+str(timeindex)

# MakeModule Parameters
module_type='my_little_pony_panel'
module_length = 1.996  # 2-up portrait Longi with 15cm additional gap
module_width = 0.991
orientation='portrait' 
tilt = 10

# Tracking Angle Calculation Parameters
hub_height = 0.5   
axis_azimuth = 180.0
roundTrackerAngleBool = True 
angledelta = 5
limit_angle = 45
backtrack = True
    
# SceneDict Parameters
gcr = 0.33   # ground cover ratio,  = module_height / pitch
albedo = 0.28  #'concrete'     # ground albedo

# Running the simulation
demo2 = RadianceObj(simulationname, path = testfolder)  # Create a RadianceObj 'object' named 'demo'
demo2.setGround(albedo) # input albedo number or material name like 'concrete'.  To see options, run this without any input.
epwfile = demo2.getEPW(37.5,-77.6) # pull TMY data for any global lat/lon
metdata = demo2.readEPW(epwfile) # read in the weather data
demo2.gendaylit(metdata, 4020)  # Noon, June 17th

path = C:\Users\sayala\Documents\RadianceScenes\demo
Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw  ... OK!


'skies\\sky2_1axisTrack4020.rad'

We are going to create a "Mycube.rad" file in the objects folder, right after we make the Module. 
This is a prism (so we use 'genbox'), that is black from the ground.rad list of materials ('black')
We are naming it 'cuteBox'
Its sides are going to be 0.5x0.5x0.5 m 
and We are going to leave its bottom surface coincident with the plane z=0, but going to center on X and Y.
The command for this description is :
>'! genbox black PVmodule 0.5 0.5 0.5 | xform -t -0.25 -0.25 0'


In [19]:
demo2.makeModule(name=module_type,x=module_width,y=module_length)   
name='Mycube'
text='! genbox black cuteBox 0.5 0.5 0.5 | xform -t -0.25 -0.25 0'
customObject = demo2.makeCustomObject(name,text)


('\nModule Name:', 'my_little_pony_panel')
REWRITING pre-existing module file. 
Module my_little_pony_panel successfully created
('\nCustom Object Name', 'objects\\Mycube.rad')


We are saving the filename and address into customObject so we can call it when making the Scene and add it to it.

At this point, you can go to the objects folder and check that the object was created properly.
### objects\\Mycube.rad

Let's create teh SCene now:

In [20]:
tracker_theta, tracker_height, azimuth_ang = demo2._getTrackingGeometryTimeIndex(metdata, timeindex=timeindex, angledelta = angledelta, roundTrackerAngleBool = roundTrackerAngleBool, axis_azimuth = axis_azimuth, limit_angle = limit_angle, backtrack = backtrack, gcr = gcr, hubheight = hub_height, module_height = module_length )
sceneDict = {'tilt':tracker_theta,'pitch': module_length / gcr,'height':tracker_height,'orientation':orientation,'azimuth':azimuth_ang, 'module_type':module_type}  
scene = demo2.makeScene(module_type, sceneDict) #makeScene creates a .rad file with 20 modules per row, 7 rows.

For this timestamp, panels are facing West
Tracker theta has been calculated to 4.722 and rounded to nearest tracking angle 5.0
Module clearance height has been calculated to 0.413, for this tracker theta.


Now let's add the customObject to the Scene. We are not going to translate it or anything because we want it at the center, but you can pass translation, rotation, and any other XFORM command from Radiance.

I am passing a rotation 0 because xform has to have something (I think) otherwise it gets confused.

In [21]:
demo2.appendtoScene(scene.radfile, customObject, '!xform -rz 0')

octfile = demo2.makeOct(demo2.getfilelist())  # makeOct combines all of the ground, sky and object files into a .oct file.

Created 1axisTrack4020.oct


At this point you should be able to go into a command window (cmd.exe) and check the geometry. Example:
    
   #### rvu -vf views\front.vp -e .01 Torque_tube_hex_test.oct
   
And then proceed happily with your analysis. If any of the sensors hits the Box object we just created, the list of materials should say something with "cuteBox" on it. (That's why I set the clearance so low for this example, so the module and the cuteBox intercept).

In [22]:
analysis = AnalysisObj(octfile, demo2.basename)  # return an analysis object including the scan dimensions for back irradiance
frontDict, backDict = analysis.analysis(octfile, demo2.basename, scene.frontscan, scene.backscan)  # compare the back vs front irradiance  
print('Annual bifacial ratio: %0.3f - %0.3f' %(min(analysis.backRatio), np.mean(analysis.backRatio)) )

Linescan in process: 1axisTrack4020_Front
Linescan in process: 1axisTrack4020_Back
Saved: results\irr_1axisTrack4020.csv
Annual bifacial ratio: 0.000 - 0.045


# Read and clean RESULTS!
When using a gap between modules (or a torque tube for 1-up), some of the materials might end up being "sky", "ground", torquetube, or even inner sides of the modules themselves if the resolution is very high.

The following is an example on how to use the read1Result and cleanResult functions of this last update.

In [2]:
resultFile='C:\Users\sayala\Documents\RadianceScenes\Plot5\\results\irr_PRISM_2UP_'+str(4240)+'_tubeZgap_0.15_GAP_0.csv'
test = read1Result(resultFile)
nmodules = 2 # 2-up configuration. 
sensorsy = 200 # this was the original resolution. Since we are removing some sensors 
               # that end up in the gap, the routine will interpolate to get this same number of sensors.
               # it will interpolate to 100 sensors per panel for the 2-up configuration for this example.
sazm = 270 # for the timestamp analyzed, the panels were facing West. This is important because the sensors and modules must be switched so it matches when the geometry is rotated...
Front4240, Back4240 = cleanResult(test, sensorsy, nmodules, sazm)

Front type materials index and occurrences: 
0  -->  98  ::  a10.3.a0.PVmodule.6457
1  -->  1  ::  a10.3.a1.PVmodule.3267
2  -->  101  ::  a10.3.a1.PVmodule.6457
Panel a0 Front material 0
Panel a1 Front material 2
Rear type materials index and occurrences: 
0  -->  100  ::  a10.3.a0.PVmodule.2310
1  -->  100  ::  a10.3.a1.PVmodule.2310
Panel a0 Rear material 0
Panel a1 Rear material 1


In [3]:
Front4240


array([705.75506667, 705.62093535, 705.48677138, 705.35260707,
       705.21847475, 705.08434343, 704.95021212, 704.81608081,
       704.68194949, 704.54781818, 704.41368687, 704.27955556,
       704.14542424, 704.01129293, 703.877133  , 703.74299697,
       703.60886566, 703.47473434, 703.34060303, 703.20647172,
       703.0723404 , 702.93820909, 702.80405185, 702.66991313,
       702.53578182, 702.40165051, 702.26751919, 702.13338788,
       701.99923266, 701.86509192, 701.73093737, 701.59679596,
       701.46266465, 701.32853333, 701.19407374, 701.57879562,
       701.73718182, 701.61652525, 701.49589764, 701.37527475,
       701.25459933, 701.13395623, 701.01333434, 700.8926798 ,
       700.77201111, 700.65134242, 700.53067374, 700.41002256,
       700.2893697 , 700.16873468, 700.04811549, 699.92746364,
       699.80679495, 699.68615724, 699.56552424, 699.44487037,
       699.32419125, 699.20351313, 699.08288283, 698.96221414,
       698.84157172, 698.72095623, 698.60014613, 698.28